# LSTM in numpy

[
    h_pre_lstm_cell = W_hh @ h_t-1 + W_hx @ x_t + b_h

    input gate  = i_t = sigmoid(h_pre_lstm_cell)  : how much/what to retain/input to the cell state of x_t, h_t-1
    forget gate = f_t = sigmoid(h_pre_lstm_cell)  : how much/what to forget of the prev cell state as a funct of x_t, h_t-1
    cell gate   = g_t = tanh(h_pre_lstm_cell) 
    output gate = o_t = sigmoid(h_pre_lstm_cell) 
]

IMPO: cell state update rule
- cell state c_t = element_wise_prod(c_t-1 * f_t) + element_wise_prod(i_t * g_t)

- element_wise_prod(c_t-1 * f_t):
cell state updated by scalindg down prev cell state acccording to f_t; f_t defines how much to forget; f_t is a funct of x_t, h_t-1 \
    if f_t = torch.ones -> all prev cell state is carried on \
    if f_t = torch.zeros -> all prev cell state dies \
with proper weight to learn forget gate rule we learn what to keep/discard from memory 
- hidden state c_t = element_wise_prod(tanh(c_t) * o_t)


pytorch implements the h_pre_lstm_cell computation using a single matrix mul and then splitting the output matrix

core idea:
- sigmoid outputs values in [0, 1]
- tanh outputs values in [-1, +1]


In [1]:
import torch
import torch.nn as nn
import numpy as np

In [40]:
BATCH_SIZE = 3
XFEATURES = 20
HIDDEN_SIZE = 100

In [41]:
model = nn.LSTMCell(XFEATURES, HIDDEN_SIZE)
print("weight_hh ", model.weight_hh.shape)
print("weight_ih ", model.weight_ih.shape)
print("bias_hh ", model.bias_hh.shape)
print("bias_ih ", model.bias_ih.shape)

weight_hh  torch.Size([400, 100])
weight_ih  torch.Size([400, 20])
bias_hh  torch.Size([400])
bias_ih  torch.Size([400])


In [42]:
def sigmoid(x):  return 1 / (1 + np.exp(-x))
def check_equal(a, b): return np.linalg.norm(a.detach().numpy() - b) < 1e-3

# nn.LSTMCell

In [ ]:
def lstm_cell(x, h, c, W_hh, W_ih, b):
    # single lstm cell == single layer of lstm
    # can process only a single timestep/element x_i of a sequence of lenght T
    i, f, g, o = np.split(W_hh@h + W_ih@x + b, 4)
    i, f, g, o = sigmoid(i), sigmoid(f), np.tanh(g), sigmoid(o)
    c_out = f * c + i * g
    h_out = o * np.tanh(c_out)
    return h_out, c_out

In [43]:
x = np.random.randn(BATCH_SIZE, XFEATURES).astype(np.float32)
h0 = np.random.randn(BATCH_SIZE, HIDDEN_SIZE).astype(np.float32) # initial hidden state
c0 = np.random.randn(BATCH_SIZE, HIDDEN_SIZE).astype(np.float32) # initial cell state

In [44]:
h_true, c_true = model(torch.tensor(x), (torch.tensor(h0), torch.tensor(c0)))
h_true.shape, c_true.shape # and indeed the out h is vector of shape (BATCH_SIZE, HIDDEN_SIZE): 1 timestep for each obs in batch

(torch.Size([3, 100]), torch.Size([3, 100]))

In [26]:
h, c = lstm_cell(x[0], h0[0], c0[0],
                 model.weight_hh.detach().numpy(),
                 model.weight_ih.detach().numpy(),
                 model.bias_hh.detach().numpy()+model.bias_ih.detach().numpy())

In [40]:
h.shape, c.shape

((100,), (100,))

In [27]:
check_equal(h_true, h)

True

# FULL nn.LSTM

In [21]:
BATCH_SIZE = 1
XFEATURES = 20
HIDDEN_SIZE = 100
SEQ_LEN = 50

In [22]:
# so with a single lstm cell we are able to process a single timestep for a whole batch of sequences, now with by replicating the nn.LSTM module
# we will be able to process a whole sequence with a single forward call
# the idea is that we are going to call step after step LSTMCell
model = nn.LSTM(XFEATURES, HIDDEN_SIZE, num_layers=1)

In [23]:
X = np.random.randn(SEQ_LEN, XFEATURES).astype(np.float32)
h0 = np.random.randn(BATCH_SIZE, HIDDEN_SIZE).astype(np.float32) # initial hidden state
c0 = np.random.randn(BATCH_SIZE, HIDDEN_SIZE).astype(np.float32) # initial cell state

In [24]:
def lstm(X, h, c, W_hh, W_ih, b):
    # returns an array containing all the hidden states at computed at each time step
    # returns only the last cell state (i.e. the cell state obtained at the last timestep of the sequence)
    H = np.zeros((SEQ_LEN, HIDDEN_SIZE))
    for t in range(SEQ_LEN):
        h, c = lstm_cell(X[t], h, c, W_hh, W_ih, b)
        H[t] = h # at each time step we store away the hidde state obtained
    return H, c


In [25]:
H, cn = lstm(X, h0[0], c0[0],
                model.weight_hh_l0.detach().numpy(),
                model.weight_ih_l0.detach().numpy(),
                model.bias_hh_l0.detach().numpy()+model.bias_ih_l0.detach().numpy())

In [26]:
# we need to manually insert batch size cuz it is required by nn.LSTM
H_true, (h_true, c_true) = model(torch.tensor(X)[:,None,:],
                                (torch.tensor(h0)[:,None,:],
                                 torch.tensor(c0)[:,None,:]))
# nn.LSTM returns both the full matrix of hiddent stats and tuple of last hidden and last cell state

In [27]:
check_equal(H_true[:,0,:], H), check_equal(c_true, cn)

(True, True)

In [45]:
H_true.shape # returns hidden states for each timestep for each sequence in the batch

torch.Size([50, 1, 100])

# Batching

In [ ]:
# for optimization reasons (contiguity in memory) we store input data for LSTM in form:
# X[Timesteps, Batch_Size, x_i_feat] # Len, N_obs, H_in from pytorch docs notation
# so we can get at the timestep t for the whole batch as: X[t,:,:]
# thus we need to change our implementations:
# X[SEQ_LEN, BATCH_SIZE, XFEATURES]
# H[SEQ_LEN, BATCH_SIZE, HIDDEN_SIZE]

In [28]:
def lstm_cell(x, h, c, W_hh, W_ih, b):
    i, f, g, o = np.split(h@W_hh + x@W_ih + b, 4, axis=1)
    i, f, g, o = sigmoid(i), sigmoid(f), np.tanh(g), sigmoid(o)
    c_out = f*c + i*g
    h_out = o * np.tanh(c_out)
    return h_out, c_out

def lstm(X, h, c, W_hh, W_ih, b):
    # returns an array containing all the hidden states at computed at each time step
    # returns only the last cell state (i.e. the cell state obtained at the last timestep of the sequence)
    H = np.zeros((SEQ_LEN, BATCH_SIZE, HIDDEN_SIZE))
    for t in range(SEQ_LEN):
        h, c = lstm_cell(X[t], h, c, W_hh, W_ih, b)
        H[t] = h # at each time step we store away the hidde state obtained
    return H, c


In [ ]:
BATCH_SIZE = 128
XFEATURES = 20
HIDDEN_SIZE = 100
SEQ_LEN = 50

In [31]:
X = np.random.randn(SEQ_LEN, BATCH_SIZE, XFEATURES).astype(np.float32)
h0 = np.random.randn(1, BATCH_SIZE, HIDDEN_SIZE).astype(np.float32) # initial hidden state
c0 = np.random.randn(1, BATCH_SIZE, HIDDEN_SIZE).astype(np.float32) # initial cell state

In [32]:
H_true, (h_true, c_true) = model(torch.tensor(X),
                                (torch.tensor(h0),
                                 torch.tensor(c0)))
# nn.LSTM returns both the full matrix of hidden states and a tuple of: (hidden state at last timestep and cell state at last timestep)

In [46]:
H_true.shape # returns hidden states for each timestep for each sequence in the batch

torch.Size([50, 1, 100])

In [36]:
H, cn = lstm(X, h0[0], c0[0],
                model.weight_hh_l0.detach().numpy().T,
                model.weight_ih_l0.detach().numpy().T,
                model.bias_hh_l0.detach().numpy()+model.bias_ih_l0.detach().numpy())

In [38]:
check_equal(H_true, H), check_equal(c_true, cn)

(True, True)